In [8]:
import json
import spotipy
import webbrowser
import datetime
import requests
import base64
from datetime import datetime
import time
from spotify_auth import re_authenticate, setup_spotify #Works well at least for re_auth
import configparser
import os
from IPython.display import display, HTML
import requests
import pyautogui
from PIL import ImageGrab
from functools import partial
import sys 
from pytesseract import pytesseract
import pandas as pd
import re

auto_km_path = r"C:\Users\sbuca\Documents\pierre\autokm"
sys.path.insert(0, auto_km_path)

from process_image import get_coordinates
from input_listener import input_detector

#ImageGrab.grab = partial(ImageGrab.grab, all_screens=True) #allows locate on screen to get second monitor

config = configparser.ConfigParser()

current_dir = os.getcwd()
path_to_tesseract = r"C:\Users\sbuca\Documents\pierre\autokm\Tesseract-OCR\tesseract.exe" #will fail in jupyter folder

pytesseract.tesseract_cmd = path_to_tesseract

#May need to screenshot monitors individually

In [9]:
### AUTHENTICATION FUNCTIONS ##
def setup_spotify(config_file_path):
    return

def check_expiration(config_file_path):
    config.read(config_file_path)
    
    token_expiration = config["TOKENS"]["expires_at"]
    
    if float(token_expiration) < float(time.mktime(datetime.now().timetuple())):
        re_authenticate()
        
        
def create_spotify_connection():
    #Must fix, must be ran twice to reauthenticate
    current_file_path = os.getcwd()
    config_file_path = current_file_path + "\\" + "spotify_config.ini"
    
    if os.path.exists(config_file_path):
        config.read(config_file_path)
        
        #Update tokens if outdated
        check_expiration(config_file_path)
        
        #Connect to spotify
        spotifyObject = spotipy.Spotify(auth=config["TOKENS"]["access_token"])
        
        return spotifyObject
        
    
    else:
        setup_spotify(config_file_path)
        create_spotify_connection()

In [10]:
### SPOTIFY MUSIC FUNCTION ###

#Must add error handling (trying to play song when already playing)
def music_controller(action, client):
    if action in ["next", "skip"]:
        client.next_track()
        
    if action in ["back", "previous"]:
        client.previous_track()
        
    if action in ["pause", "stop"]:
        client.pause_playback()
        
    if action in ["resume"]:
        client.start_playback()
        
    if action == "shuffle":
        client.shuffle(True)

    if action == "replay":
        client.previous_track()
        client.next_track()
        
        
#Could add functioinality when you say next playlist and it will move to second choice
def find_best_playlist(playlists, target, client, track_artist):
    playlist_likes = dict()

    #Loop through all playlistd
    for playlist in playlists['playlists']['items']:
        
        playlist_name = playlist['name'].lower()
        playlist_id = playlist['id']
        playlist_creator = playlist['owner']['id'].lower()

        #Filter for non spotify playlists (They generally suck - too many features included)
        if playlist_creator == "spotify":
            continue
            
        if fuzz.ratio(playlist_name, target) <= 70 or fuzz.partial_ratio(playlist_name, target) <= 70:
            continue
            
        playlist_followers = client.user_playlist(user=playlist_creator, playlist_id=playlist_id)["followers"]["total"]

        playlist_likes[playlist_followers] = playlist_id

            
    return playlist_likes[max(playlist_likes)]


def find_best_song(tracks, target, track_artist, client):
    track_popularity = dict()
    
    for track in tracks['tracks']['items']:
        
        #To get specific song like no flocking by kodak
        artist = track['artists'][0]['name'].lower()
        track_name = track['name'].lower()
        track_id = track['id']
        
        
        if fuzz.ratio(target, track_name) < 70 or fuzz.ratio(track_artist, artist):
            if fuzz.partial_ratio(track_artist, artist) < 70:
                continue
            
        track_popularity[track['popularity']] = track_id
        
        
    return track_popularity[max(track_popularity)]


def find_best_album(albums, target, album_artist, client):
    album_score = dict()
    
    for album in albums['albums']['items']:
        
        album_name = album['name'].lower()
        album_id = album['id']
        artist = album['artists'][0]['name'].lower()

        album_fuzz = fuzz.ratio(target, album_name)
        artist_fuzz = fuzz.ratio(album_artist, artist)
        artist_pfuzz = fuzz.partial_ratio(album_artist, artist)
        
        
        if album_fuzz < 70 or artist_fuzz < 70:
            if artist_pfuzz < 70:
                continue
                
        album_score[album_fuzz] = album_id
                
    return album_score[max(album_score)]   
            
            
            
def search_music(command, track_type, client, track_artist=None):
    if track_type == "song":
        track_type = "track"
        
    command = command.lower()
    
    if track_artist is not None:
        track_artist = track_artist.lower()
    
    results = client.search(q=command, type=track_type)

    
    if track_type == "playlist":
        return find_best_playlist(results, command, client, track_artist)
    
    if track_type in ["track","song"]:
        return find_best_song(results, command, track_artist, client)
    
    if track_type in ["album"]:
        return find_best_album(results, command, track_artist, client)
    

def get_command_type(command):
    pattern = re.compile(r"(.+) by (.+)")
    match = pattern.search(command)
    
    song_name = match.group(1)
    artist_name = match.group(2)
    
    return song_name, artist_name


def activate_device(client):

     
    
#command will be like "No Flockin by Kodak Black"
#play key word will not be in there because we will remove it before we route this command
def music_player(command, client):
    if "playlist" in command:
        command = command.replace(" playlist", "")
        id_ = search_music(command, "playlist", client)
        
        #idk why its different, this method is eaier but this method dont work for songs
        client.start_playback(context_uri=("spotify:playlist:"+id_))
        
    
    #Searchs for song or albums    
    else:
        track, artist = get_command_type(command)
        
        if "album" in track:
            track = track.replace(" album", "")
            
            id_ = search_music(track, "album", client, artist)
            client.start_playback(context_uri=("spotify:album:"+id_))
            
        else:
            id_ = search_music(track, "track", client, artist)
            client.start_playback(uris=[('spotify:track:' + str(id_))])

In [6]:
find_best_album(results, "No Flockin", "", spotify_client)

NameError: name 'results' is not defined

In [7]:
results = spotify_client.search(q="No Flockin", type="track,album,playlist")

NameError: name 'spotify_client' is not defined

In [391]:
for _ in results["playlists"]['items']:
    print(_['name'])

No Flockin'
No flockin’ 
No Flockin'💯
no flockin js blockin
No Flockin'
No Flockin'
No Flockin'
No Flockin'
If She Not Flockin We Not Talkin
no flockin 


In [388]:
results

{'albums': {'href': 'https://api.spotify.com/v1/search?query=No+Flockin&type=album&offset=0&limit=10',
  'items': [{'album_type': 'single',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/46SHBwWsqBkxI7EeeBEQG7'},
      'href': 'https://api.spotify.com/v1/artists/46SHBwWsqBkxI7EeeBEQG7',
      'id': '46SHBwWsqBkxI7EeeBEQG7',
      'name': 'Kodak Black',
      'type': 'artist',
      'uri': 'spotify:artist:46SHBwWsqBkxI7EeeBEQG7'}],
    'available_markets': ['AD',
     'AG',
     'AL',
     'AM',
     'AO',
     'AR',
     'AT',
     'AU',
     'AZ',
     'BA',
     'BB',
     'BD',
     'BE',
     'BG',
     'BI',
     'BJ',
     'BO',
     'BR',
     'BS',
     'BT',
     'BW',
     'BZ',
     'CA',
     'CD',
     'CG',
     'CH',
     'CI',
     'CL',
     'CM',
     'CO',
     'CR',
     'CV',
     'CW',
     'CY',
     'CZ',
     'DE',
     'DJ',
     'DK',
     'DM',
     'DO',
     'DZ',
     'EC',
     'EE',
     'ES',
     'ET',
     'FI',
     '

In [376]:
results.keys()

dict_keys(['albums', 'tracks', 'playlists'])

In [370]:
    for album in results['albums']['items']:
        
        album_name = album['name'].lower()
        album_id = album['id']
        artist = album['artists'][0]['name'].lower()

In [11]:
spotify_client = create_spotify_connection()

In [12]:
spotify_client

In [359]:
music_player("heros and villians by metro boomin", spotify_client)

In [20]:
spotify_client = create_spotify_connection()
spotify_client = create_spotify_connection()
#x = music_player("Her Loss by Drake", spotify_client)
devices = spotify_client.devices()
#Must fix for certain playlists being albums

In [21]:
devices['devices']

[{'id': '132fddb7c9d835c45cd27cc14b71bcf235713a8d',
  'is_active': False,
  'is_private_session': False,
  'is_restricted': False,
  'name': 'Web Player (Chrome)',
  'type': 'Computer',
  'volume_percent': 51},
 {'id': '88310545629f44b38748a461c86e5fc2679b828d',
  'is_active': True,
  'is_private_session': False,
  'is_restricted': False,
  'name': 'SEBASTIAN-PC',
  'type': 'Computer',
  'volume_percent': 72}]

In [290]:
fuzz.partial_ratio("Youngboy", "Youngboy Never Broke Again")

100

In [ ]:
6vrBmDYgfFotABKDg6jC94

In [ ]:
"spotify:playlist:"

In [ ]:
'spotify:track:'

In [ ]:
spotify_client.start_playback(uris=[('spotify:track:' + str(id_))])

In [ ]:
spotify_client.start_playback(uris=[('spotify:track:' + str(id_))])

In [303]:
x = spotify_client.search(q="Her Loss", type="album")

In [317]:
for _ in x['albums']['items']:
    #print(_['name'])
    print(_['artists'][0]['name'])
    print(_['id'])
    break

Drake
5MS3MvWHJ3lOZPLiMxzOU6


In [309]:
x

{'albums': {'href': 'https://api.spotify.com/v1/search?query=Her+Loss&type=album&offset=0&limit=10',
  'items': [{'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
      'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
      'id': '3TVXtAsR1Inumwj472S9r4',
      'name': 'Drake',
      'type': 'artist',
      'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/1URnnhqYAYcrqrcwql10ft'},
      'href': 'https://api.spotify.com/v1/artists/1URnnhqYAYcrqrcwql10ft',
      'id': '1URnnhqYAYcrqrcwql10ft',
      'name': '21 Savage',
      'type': 'artist',
      'uri': 'spotify:artist:1URnnhqYAYcrqrcwql10ft'}],
    'available_markets': ['AD',
     'AE',
     'AG',
     'AL',
     'AM',
     'AO',
     'AR',
     'AT',
     'AU',
     'AZ',
     'BA',
     'BB',
     'BD',
     'BE',
     'BF',
     'BG',
     'BH',
     'BI',
     'BJ',

In [275]:
spotify_client.start_playback(context_uri="spotify:playlist:6vrBmDYgfFotABKDg6jC94")

In [ ]:
def extract_

In [229]:
import re

string = "Best of Kodak Black"

# Define the regular expression pattern
pattern = re.compile(r"(.+) by (.+)")

# Search for a match in the string
match = pattern.search(string)

# Extract the song name and artist name from the match
song_name = match.group(1)
artist_name = match.group(2)

print("Song name:", song_name)
print("Artist name:",artist_name)

AttributeError: 'NoneType' object has no attribute 'group'

In [243]:
spotify_client.start_playback(uris=['spotify:track:34oWbFBfGEElvgO0a5c9V4'])

In [230]:
match

In [215]:
zxc = search_music("No Flockin", "track", "Kodak Black", spotify_client)

no flockin' kodak black 34oWbFBfGEElvgO0a5c9V4 76
no flockin' kodak black 2Q6BoynOBtVmQcKzkNZkg3 43
no flockin kodak black 4dGKPHDfhRMJDWlOluM5If 20
no flockin kodak black 0ORJ7Yx45MjkTZXtcc99R3 12


In [172]:
for _ in zz['tracks']['items']:
    xxxx = (_)
    break

In [179]:
xxxx.keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [219]:
xxxx

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/46SHBwWsqBkxI7EeeBEQG7'},
    'href': 'https://api.spotify.com/v1/artists/46SHBwWsqBkxI7EeeBEQG7',
    'id': '46SHBwWsqBkxI7EeeBEQG7',
    'name': 'Kodak Black',
    'type': 'artist',
    'uri': 'spotify:artist:46SHBwWsqBkxI7EeeBEQG7'}],
  'available_markets': ['AD',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BG',
   'BI',
   'BJ',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BZ',
   'CA',
   'CD',
   'CG',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',
   'DE',
   'DJ',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'ES',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GA',
   'GB',
   'GD',
   'GE',
   'GH',
   'GM',
   'GN',
   'GQ',
   'GR',
   'GT',
   'GW',
   'GY',
   'HK',
   'HN',
   'HR',
   'HT',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IQ',

In [186]:
artist = xxxx['artists'][0]['name']
track_name = xxxx['name']
track_id = xxxx['id']

In [187]:
print(artist,track_name,track_id)

Kodak Black No Flockin' 34oWbFBfGEElvgO0a5c9V4


In [397]:
spotify_client.devices()

HTTP Error for GET to https://api.spotify.com/v1/me/player/devices with Params: {} returned 401 due to Permissions missing


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/me/player/devices:
 Permissions missing, reason: None

In [149]:
search_music("Best of Kodak Black", "playlist", spotify_client)

'6vrBmDYgfFotABKDg6jC94'

In [116]:
from thefuzz import fuzz

target = 'Best of Kodak Black'
peepoo = 'This Is Kodak Black'

In [117]:
fuzz.partial_ratio(target,peepoo)

76

In [118]:
fuzz.ratio(target,peepoo)

74

In [36]:
music_controller("pause", spotify_client)

In [242]:
spotify_client = create_spotify_connection()

In [164]:
spotify_client.previous_track()

HTTP Error for POST to https://api.spotify.com/v1/me/player/previous with Params: {} returned 401 due to The access token expired


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/me/player/previous:
 The access token expired, reason: None

In [11]:
spotify_client.shuffle(True)

In [17]:
spotify_client.repeat('track')

In [46]:
x = spotify_client.search(q="Best of Kodak", type="playlist")

In [45]:
spotify_client.start_playback(context_uri='spotify:playlist:37i9dQZF1DZ06evO0H1BQm')

In [88]:
for i, playlist in enumerate(x['playlists']['items']):
    print(playlist['name'])

the best of kodak🖤
Best of Kodak Black
Best of Kodak Black
Best of 21 Savage & Kodak Black
Best of Babyface Ray, Veeze, EST Gee, Icewear Vezzo, Pooh Shiesty, Kodak Black,Detroit Rap,Hood Shit
This Is Kodak Black
This Is Kota the Friend
This Is Kodaline
This Is Koffee
This Is Dokken


In [120]:
x['playlists']['items'][-1]

{'collaborative': False,
 'description': 'This is Dokken. The essential tracks, all in one playlist.',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO016Nmh'},
 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DZ06evO016Nmh',
 'id': '37i9dQZF1DZ06evO016Nmh',
 'images': [{'height': None,
   'url': 'https://thisis-images.scdn.co/37i9dQZF1DZ06evO016Nmh-large.jpg',
   'width': None}],
 'name': 'This Is Dokken',
 'owner': {'display_name': 'Spotify',
  'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'primary_color': None,
 'public': None,
 'snapshot_id': 'Mjc4ODc5MzEsMDAwMDAwMDBkYTIwMjU1ZDhjNmY2Yzc1MGEwZTNiMzhkYjg4Nzk5Mg==',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DZ06evO016Nmh/tracks',
  'total': 50},
 'type': 'playlist',
 'uri': 'spotify:playlist:37i9dQZF1DZ06evO016Nmh'}

In [65]:
spotify_client.playlist_items("6vrBmDYgfFotABKDg6jC94")

{'href': 'https://api.spotify.com/v1/playlists/6vrBmDYgfFotABKDg6jC94/tracks?offset=0&limit=100&additional_types=track%2Cepisode',
 'items': [{'added_at': '2017-09-02T10:03:40Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/roderickspotify'},
    'href': 'https://api.spotify.com/v1/users/roderickspotify',
    'id': 'roderickspotify',
    'type': 'user',
    'uri': 'spotify:user:roderickspotify'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/21WS9wngs9AqFckK7yYJPM'},
       'href': 'https://api.spotify.com/v1/artists/21WS9wngs9AqFckK7yYJPM',
       'id': '21WS9wngs9AqFckK7yYJPM',
       'name': 'PnB Rock',
       'type': 'artist',
       'uri': 'spotify:artist:21WS9wngs9AqFckK7yYJPM'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/46SHBwWsqBkxI7EeeBEQG7'},
       'href': 'https://api.spotify.com/v1/artists

In [67]:
spotify_client.user_playlist("6vrBmDYgfFotABKDg6jC94")

HTTP Error for GET to https://api.spotify.com/v1/users/6vrBmDYgfFotABKDg6jC94/starred with Params: {} returned 403 due to You don't have permission to perform this action.


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/users/6vrBmDYgfFotABKDg6jC94/starred:
 You don't have permission to perform this action., reason: None

In [95]:
pol = spotify_client.user_playlist(user="roderickspotify", playlist_id="6vrBmDYgfFotABKDg6jC94")

In [101]:
pol["followers"]["total"]

4017

In [81]:
max({325325:"y", 13251356234:"n"})

13251356234

In [100]:
pol

{'collaborative': False,
 'description': 'List created without access to all &#x27;&#x27;best songs.&#x27;&#x27; Unfortunately some songs are only on YT',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/6vrBmDYgfFotABKDg6jC94'},
 'followers': {'href': None, 'total': 4017},
 'href': 'https://api.spotify.com/v1/playlists/6vrBmDYgfFotABKDg6jC94?additional_types=track',
 'id': '6vrBmDYgfFotABKDg6jC94',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab67706c0000bebbff9942fd00e437e885da93d5',
   'width': None}],
 'name': 'Best of Kodak Black',
 'owner': {'display_name': 'Roderick VB',
  'external_urls': {'spotify': 'https://open.spotify.com/user/roderickspotify'},
  'href': 'https://api.spotify.com/v1/users/roderickspotify',
  'id': 'roderickspotify',
  'type': 'user',
  'uri': 'spotify:user:roderickspotify'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MTM2LGQ2NzQ4ZDFlYWU5OGQ1YzRiMjQzMjY5MzI1NWU2NTBhMDU0ZDU4MDY=',
 'tracks': {'href': 'https://